In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
device = "cuda"
import pickle
import torch
import numpy as np
from PIL import Image
from pathlib import Path
fm = Path("/data/natsuki/training116/00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/network-snapshot-011289.pkl")
assert fm.is_file()
with open(fm, 'rb') as f:
    G = pickle.load(f)['G_ema'].to(device)
def mapping(seed=1, psi=1):
    label = torch.zeros([1, G.c_dim], device=device)
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    w = G.mapping(z, label, truncation_psi=psi)
    return w
def synthesis(w, synth=True):
    if synth:
        synth_image = G.synthesis(w, noise_mode='const')
    else:
        synth_image = w
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    img = Image.fromarray(synth_image, 'RGB')
    return img

In [ ]:
w1 = mapping(43076)
w2 = mapping(28481)

q = 8
a = torch.vstack([w1[0,:q,:], w2[0,q:,:]])[None,:,:]
synthesis(a)

In [ ]:
p = 0.7
a = w1*p+w2*(1-p)
synthesis(a)

In [ ]:
# 中間特徴量の可視化
from random import randint

import cv2

def forward(self, ws, **block_kwargs):
    block_ws = []
    ws = ws.to(torch.float32)
    w_idx = 0
    for res in self.block_resolutions:
        block = getattr(self, f'b{res}')
        block_ws.append(ws.narrow(1, w_idx, block.num_conv + block.num_torgb))
        w_idx += block.num_conv
    x = img = None
    imgs = list()
    for res, cur_ws in zip(self.block_resolutions, block_ws):
        block = getattr(self, f'b{res}')
        x, img = block(x, img, cur_ws, **block_kwargs)
        imgs.append(img)
    return imgs

imgs = forward(G.synthesis, mapping(randint(1, 10000000)), noise_mode='const')
def eqhistlat(imgs, i=0):
    img = imgs[-i-1]
    a = max(img.min().abs(), img.max().abs())
    synth_image = (img/a + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

    equ = [cv2.equalizeHist(synth_image[:,:,i]) for i in range(3)]
    equ = np.stack(equ, axis=2)
    equ = cv2.resize(equ, dsize=None, fx=2**i, fy=2**i)
    return equ

equs = [eqhistlat(imgs, i) for i in range(3)]
equs = np.hstack(equs)
Image.fromarray(equs)